In [ ]:
%pip install pytesseract
# 必须装 tesseract-lang

In [1]:
import pytesseract
from PIL import Image,ImageDraw
import easyocr

In [31]:
def cencos(image_path,out_path,box):
    
    image=Image.open(image_path)

    draw=ImageDraw.Draw(image)

    draw.rectangle(box,fill='white')

    image.save(out_path)


    

In [14]:
image=Image.open('gml.png')
rst=pytesseract.image_to_string(image=image,lang='chi_sim')
print(rst)

In [ ]:
# cnorc
%pip install 'cnocr[ort-cpu]'

In [ ]:
import cnocr
# https://cnocr.readthedocs.io/zh/latest/usage/

orc=cnocr.CnOcr()
info=orc.ocr(image)

info

In [14]:
import numpy as np
position=info[3]['position']

for index,data in enumerate(info):
    if index==3:continue
    position=data['position'].astype('int').tolist()
    box=[*position[0],*position[2]]
    try:
        cencos('copy.png','copy.png',box)
    except Exception:
        print(index,data['text'])

4 用于感冒引起的头痛,发热,鼻塞,流涕，咽痛


In [ ]:
%pip install easyocr

In [58]:
'''
    [335, 256], [975, 256], [975, 497], [335, 497]
'''
def find_max_height(text):
    ret=-1
    val=-1
    for idx,info in enumerate(text):
        position,*_=info
        top,bottom,*_=position
        height=bottom[0]-top[0]
        if val < height:
            ret=idx
            val=height
    return ret,val

def apply_mosaic(image,area,block_size):
    x1, y1, x2, y2 = area
    
    # 对给定区域进行马赛克处理
    for y in range(y1, y2, block_size):
        for x in range(x1, x2, block_size):
            box = (x, y, min(x2, x + block_size), min(y2, y + block_size))
            color = image.crop(box).getcolors()[0][1]
            image.paste(color, box)
    
 
    
def masaic(path):
    reader=easyocr.Reader(lang_list=['ch_sim'])
    text=reader.readtext(path)
    idx,_=find_max_height(text)
    temp=Image.open(path)
    for index,data in enumerate(text):
        position,*info=data
        if index==idx:continue
        box=[e.astype('int') for e in (*position[0],*position[2])]
        try:
            apply_mosaic(temp,box,10)
        except Exception as e:
            e.with_traceback()
            print(e,index,info[0])
    
    temp.save('temp.png')

In [61]:
masaic('gml.png')